In [ ]:
# Modeling exam
# Vadim Zhovtanyuk
# Cisco Systems, 2020

In [1]:
import numpy as np
import pandas as pd
import scipy.stats
import statistics as stat
from matplotlib import pyplot
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import sys

In [2]:
#Loading the data into pandas dataframe
file = 'dnac-customer-bookings.csv'
data = pd.read_csv(file, sep=',', header=0, parse_dates=True)

In [3]:
#First look at the data
data.head(5)

,cust,segment,vertical,sub_vertical,country,bookings,purchase
0,d66648938c68318a8dad17a77f0c8327,Commercial,Manufacturing,Hi-tech/Computers Manufacturing,UNITED STATES,103356,False
1,dcce1fa11f27a6a2c57c8aacf5f0219c,Commercial,Retail,Other (Retail),UNITED STATES,62952,True
2,136b88d1b2aea8ae74cbfe111830af69,Commercial,Retail,Other (Retail),ITALY,2886,True
3,674c6fcb90b5d29765f266007ad423e1,Enterprise,Retail,Other (Retail),UNITED STATES,80253,True
4,97c72b3833ce8ef5154826fc128522b1,Commercial,Health Care,Providers,AUSTRALIA,29424,False


In [4]:
#Removing cust column which is unique in each row and hence not useful for further analysis
data = data.drop("cust", axis=1)

In [5]:
#Checking if there are any "0" values
data.isnull().any()

segment         False
vertical        False
sub_vertical    False
country         False
bookings        False
purchase        False
dtype: bool

In [6]:
#Checking "segment" column
data["segment"].value_counts()

Enterprise             16623
Commercial             10471
commercial               246
enterprise                 2
enterprise customer        1
Name: segment, dtype: int64

In [7]:
#Fixing different names for the same segment
def fix(column):
    results = {}
    results['column'] = column.name

    #Replacing the "wrong" names
    segment_column = []
    for value in column:
        if value == "enterprise customer":
            segment_column.append("Enterprise")
        elif value == "enterprise":
            segment_column.append("Enterprise")
        elif value == "commercial":
            segment_column.append("Commercial")
        else:
            segment_column.append(value)

    #Creating new column data with 'fixed' entries
    results['data'] = segment_column

    return results

#Replacing data with new value
missing_data = data.apply(fix)
for m in missing_data:
    data[m['column']] = m['data']

In [8]:
#Verification of segment name
data["segment"].value_counts()

Enterprise    16626
Commercial    10717
Name: segment, dtype: int64

In [9]:
#List of segment names
data['segment'].sort_values().unique()

array(['Commercial', 'Enterprise'], dtype=object)

In [10]:
#Encoding segment column to prepare data for further analysis 
data["segment"] = data["segment"].astype("category")
segment_encode = LabelEncoder()
data["segment"] = segment_encode.fit_transform(data.segment)
segment_one_hot = OneHotEncoder()
segment_one_hot_encode = segment_one_hot.fit_transform(data.segment.values.reshape(-1,1)).toarray()
ohe_variable = pd.DataFrame(segment_one_hot_encode, columns = ["segment_" + str(int(i)) for i in range (segment_one_hot_encode.shape[1])])
data = pd.concat([data, ohe_variable], axis=1)
data = data.drop("segment", axis=1)

In [11]:
#Checking "vertical" column
data['vertical'].value_counts()

Professional Services     5553
Manufacturing             5112
Government                3559
Retail                    3483
Technical Services        2816
Financial Services        2573
Wholesale/Distribution    2384
Health Care               1863
Name: vertical, dtype: int64

In [12]:
#List of vertical names
data['vertical'].sort_values().unique()

array(['Financial Services', 'Government', 'Health Care', 'Manufacturing',
       'Professional Services', 'Retail', 'Technical Services',
       'Wholesale/Distribution'], dtype=object)

In [13]:
#Encoding "vertical" column to prepare data for further analysis 
data["vertical"] = data["vertical"].astype("category")
vertical_encode = LabelEncoder()
data["vertical"] = vertical_encode.fit_transform(data.vertical)
vertical_one_hot = OneHotEncoder()
vertical_one_hot_encode = vertical_one_hot.fit_transform(data.vertical.values.reshape(-1,1)).toarray()
ohe_variable = pd.DataFrame(vertical_one_hot_encode, columns = ["vertical_" + str(int(i)) for i in range (vertical_one_hot_encode.shape[1])])
data = pd.concat([data, ohe_variable], axis=1)
data = data.drop("vertical", axis=1)

In [14]:
#Checking "sub_vertical" column
data["sub_vertical"].value_counts()

Federal/Central Government                          3559
Other (Technical Services)                          2816
Other (Credit Card, mortgage, financing, etc.)      2573
Other (Retail)                                      2472
Wholesale/Distribution                              2384
Other (Professional Services)                       1908
Providers                                           1863
Discrete Manufacturing                              1834
Consumer Products Manufacturing                     1703
Hi-tech/Computers Manufacturing                     1575
Consulting Non-Technical                            1125
Construction/Engineering                            1013
Consumer Services(dry cleaners,video rental,etc)    1011
Real Estate                                          785
Membership organizations                             722
Name: sub_vertical, dtype: int64

In [15]:
#List of sub_vertical names
data["sub_vertical"].sort_values().unique()

array(['Construction/Engineering', 'Consulting Non-Technical',
       'Consumer Products Manufacturing',
       'Consumer Services(dry cleaners,video rental,etc)',
       'Discrete Manufacturing', 'Federal/Central Government',
       'Hi-tech/Computers Manufacturing', 'Membership organizations',
       'Other (Credit Card, mortgage, financing, etc.)',
       'Other (Professional Services)', 'Other (Retail)',
       'Other (Technical Services)', 'Providers', 'Real Estate',
       'Wholesale/Distribution'], dtype=object)

In [16]:
#Encoding "sub_vertical" column to prepare data for further analysis 
data["sub_vertical"] = data["sub_vertical"].astype("category")
sub_vertical_encode = LabelEncoder()
data["sub_vertical"] = sub_vertical_encode.fit_transform(data.sub_vertical)
data
sub_vertical_one_hot = OneHotEncoder()
sub_vertical_one_hot_encode = sub_vertical_one_hot.fit_transform(data.sub_vertical.values.reshape(-1,1)).toarray()
ohe_variable = pd.DataFrame(sub_vertical_one_hot_encode, columns = ["sub_vertical_" + str(int(i)) for i in range (sub_vertical_one_hot_encode.shape[1])])
data = pd.concat([data, ohe_variable], axis=1)
data = data.drop("sub_vertical", axis=1)

In [17]:
#Checking "country" column
data["country"].value_counts()

UNITED STATES     13810
UNITED KINGDOM     2246
CHINA              1836
GERMANY            1683
CANADA             1277
JAPAN              1228
AUSTRALIA          1183
FRANCE              983
ITALY               787
MEXICO              733
INDIA               705
BRAZIL              451
USA                 347
DEUTSCHLAND          74
Name: country, dtype: int64

In [18]:
#Fixing different names for the same country issues
def fix(column):
    results = {}
    results['column'] = column.name

    #Replacing different names for the same country
    country_column = []
    for value in column:
        if value == "USA":
            country_column.append("UNITED STATES")
        elif value == "DEUTSCHLAND":
            country_column.append("GERMANY")
        else:
            country_column.append(value)

    #Creating new column data with 'fixed' entries
    results['data'] = country_column

    return results  

#Replacing data with new value
temp_data = data.apply(fix)
for m in temp_data:
    data[m['column']] = m['data']

In [19]:
#Verification of country names
data["country"].value_counts()

UNITED STATES     14157
UNITED KINGDOM     2246
CHINA              1836
GERMANY            1757
CANADA             1277
JAPAN              1228
AUSTRALIA          1183
FRANCE              983
ITALY               787
MEXICO              733
INDIA               705
BRAZIL              451
Name: country, dtype: int64

In [20]:
#List of countrie names
data["country"].sort_values().unique()

array(['AUSTRALIA', 'BRAZIL', 'CANADA', 'CHINA', 'FRANCE', 'GERMANY',
       'INDIA', 'ITALY', 'JAPAN', 'MEXICO', 'UNITED KINGDOM',
       'UNITED STATES'], dtype=object)

In [21]:
#Encoding "country" column to prepare data for further analysis 
data["country"] = data["country"].astype("category")
country_encode = LabelEncoder()
data["country"] = country_encode.fit_transform(data.country)
country_one_hot = OneHotEncoder()
country_one_hot_encode = country_one_hot.fit_transform(data.country.values.reshape(-1,1)).toarray()
ohe_variable = pd.DataFrame(country_one_hot_encode, columns = ["country_" + str(int(i)) for i in range (country_one_hot_encode.shape[1])])
data = pd.concat([data, ohe_variable], axis=1)
data = data.drop("country", axis=1)

In [22]:
#Checking "bookings"
data["bookings"].value_counts()

unknown    73
2406        8
2272        7
2028        7
1476        7
           ..
102593      1
192458      1
18208       1
27961       1
8300        1
Name: bookings, Length: 22474, dtype: int64

In [23]:
#Replacing "unknown" and negative values with pre-defined value "1010101"
def fix(column):
    results = {}
    results['column'] = column.name

    fix_column = []
    for value in column:
        if value == "unknown":
            fix_column.append("1010101")
        elif str(value).startswith("-"):
            fix_column.append("1010101")
        else:
            fix_column.append(value)

    #Creating new column data with 'fixed' entries
    results['data'] = fix_column

    return results  

#Replacing data with new value
temp_data = data.apply(fix)
for m in temp_data:
    data[m['column']] = m['data']

In [24]:
#Verification of bookings values
data["bookings"].value_counts()

1010101    75
2406        8
1795        7
1476        7
2028        7
           ..
192458      1
18208       1
27961       1
3094        1
8300        1
Name: bookings, Length: 22472, dtype: int64

In [25]:
#Fixing wrong data in bookings 
def missing(column):
    results = {}
    results['column'] = column.name

    #Step1 -  Removing wrong entries and calcualting median 
    s1_column = [x for x in column if x != 1010101]
    results['removed_median'] = stat.median(s1_column)
    
    #Step2 - Replace wrong entries with (removed) median
    s2_column = []
    for value in column:
        if value != 1010101:
            s2_column.append(value)
        else:
            s2_column.append(results['removed_median'])

    #Creating new column data with 'fixed' entries
    results['data'] = s2_column

    return results

#Replace missing data with mean value
ds = data.astype({'bookings': 'int64'})
missing_data = ds.select_dtypes(include=['float64', 'int64']).apply(missing)
for m in missing_data:
    ds[m['column']] = m['data']  
    
data = ds.astype({'bookings': 'int64'})    

In [26]:
data.groupby("purchase").size()

purchase
False    17743
True      9600
dtype: int64

In [27]:
#Purchase list booleans
data["purchase"].sort_values().unique()

array([False,  True])

In [28]:
#Encoding "purchase" column to prepare data for further analysis 
data["purchase"] = data["purchase"].astype("category")
purchase_encode = LabelEncoder()
data["purchase"] = purchase_encode.fit_transform(data.purchase)

In [29]:
data.head(5)

,bookings,purchase,segment_0,segment_1,vertical_0,vertical_1,vertical_2,vertical_3,vertical_4,vertical_5,...,country_2,country_3,country_4,country_5,country_6,country_7,country_8,country_9,country_10,country_11
0,103356,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,62952,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2886,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,80253,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,29424,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#data.dtypes
#data.shape
data.describe()

In [ ]:
data.corr(method='pearson')
correlations = data.corr()
fig = pyplot.figure() 
ax = fig.add_subplot(111) 
cax = ax.matshow(correlations, vmin=-1, vmax=1) 
fig.colorbar(cax) 
ticks = np.arange(0,9,1) 
ax.set_xticks(ticks) 
ax.set_yticks(ticks) 
pyplot.show()

In [ ]:
data.skew()

In [30]:
#Create feature and target for classification
features = data.drop("purchase", axis=1).values
target = data["purchase"].values

In [31]:
#Split the data for training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, random_state = 42, stratify = target)

In [35]:
#Initializing the kNN classifier with X neighbors
#Played from 3 to 100 and the improvment form 50 to 100 is small so left 50
knn_classifier = KNeighborsClassifier(n_neighbors=100)

In [36]:
#Fitting the kNN classifier on the training data
knn_classifier.fit(X_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=100, p=2,
                     weights='uniform')

In [37]:
#Extracting the accuracy score from the test sets for kNN classifiier
knn_classifier.score(X_test, y_test)

0.6450079239302694

In [38]:
#Initializing an logistic regression object 
logistic_regression = linear_model.LogisticRegression()

In [39]:
#Fitting the model to the training and test sets
logistic_regression.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
#Accuracy score of the logistic regression model 
logistic_regression.score(X_test, y_test)

0.6489089357552115

In [43]:
#Building the model with L1 penality
logistic_regression = linear_model.LogisticRegression(penalty='l2')

In [49]:
#Using GridSearchCV to search for the best parameter
grid = GridSearchCV(logistic_regression, {'C':[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]})
grid.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=0.0001, class_weight=None,
                                          dual=False, fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [50]:
#Print out the best parameter
grid.best_params_

{'C': 1e-05}

In [54]:
#Initializing an logistic regression object
logistic_regression = linear_model.LogisticRegression(C = 1e-05, penalty='l2')

In [55]:
#Fitting the model to the training and test sets 
logistic_regression.fit(X_train, y_train)

LogisticRegression(C=1e-05, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
#Accuracy score of the logistic regression model
logistic_regression.score(X_test, y_test)

0.6489089357552115

In [60]:
target

array([0, 1, 1, ..., 0, 1, 0])

In [ ]:
# [END]